# Bert Test (with translation)

## Import librares for notebook.

In [2]:
import torch
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
from googletrans import Translator
import pandas as pd
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity

## Import and translate data

In [20]:
#Import senator csv from senator_data_file
senators = pd.read_csv(r'/Users/jmlunamugica/code/jomilu93/sivico/senators_data.csv', converters={'initiative_list': pd.eval})

In [21]:
#Split the concatenated initiatives for each senator into bites smaller than 5000 to be able to translate.
senators["initiatives_summary_dummy_split"] = ""

n = 4999

for i, row in senators.iterrows():
    if not row["initiative_list"] == []:
        initiatives_split = [row["initiatives_summary_dummy"][i:i+n] for i in range(0, len(row["initiatives_summary_dummy"]), n)]
        senators.at[i, "initiatives_summary_dummy_split"] = initiatives_split
    else:
        senators.at[i, "initiatives_summary_dummy_split"] = []

In [22]:
#Translate the concatenated, split string of initiatives per senator, store in new column.

senators["initiatives_summary_dummy_split_en"] = ""

for i, row in senators.iterrows():
#     print(f"Working on row {i} of {len(senators)}")
    if len(row["initiatives_summary_dummy_split"]) >= 1:
        en_initiatives = GoogleTranslator(source='es', target='en').translate_batch(row["initiatives_summary_dummy_split"])
        senators.at[i, "initiatives_summary_dummy_split_en"] = en_initiatives
#         print(f"Summary {i} translated successfully. Sample: {en_initiatives[0][:20]}")
    else:
#         print(f"Senator number {i}, senator {row['senadores']} has no initiatives to translate.")
        continue

In [24]:
#Join all split, translated initiatives into one long english string per senator.

senators["initiatives_summary_dummy_en"] = senators["initiatives_summary_dummy_split_en"].apply(lambda x: "".join(x))

In [25]:
senators["initiatives_summary_dummy_en"][0]

'It intends to guarantee members of the Mexican Foreign Service, in a higher percentage than that established in current legislation, access to state representation positions such as Ambassador or Consul. It proposes to establish greater clarity in the subdivision of the ordinary commissions of Education, Culture and Science and Technology, suppressing the only current denomination of the ordinary commission of "Education, Culture, Science and Technology". It proposes establishing that the Institute of Expert Services and Forensic Sciences will be the technical support area for the Prosecutor\'s Office that will have with the objective of strengthening its institutional capacity to carry out its functions in a specialized and scientific manner. It determines the establishment of an Expert Career Service. It proposes to establish that citizen initiatives have the preferential character. It proposes to establish that the National Educational System will consider education in terms of pri

## Feed text through BERT model to create a summary.

In [26]:
#Import BERT model
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
senators["initiative_summary_en"] = ""

In [19]:
#     print(f"Working on summary {i} of {len(senators)}")
sentences = sent_tokenize(senators["initiatives_summary_dummy_en"][37])

initiatives_split = [row["initiatives_summary_dummy"][i:i+n] for i in range(0, len(row["initiatives_summary_dummy"]), n)]

print(f"This senator has {len(sentences)} sentences.")
tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]
print("Finished sentence tokenization")
max_len = 0
for i in tokenized_sentences:
    if len(i) > max_len:
        max_len = len(i)

padded_sentences = []
for i in tokenized_sentences:
    while len(i) < max_len:
        i.append(0)
    padded_sentences.append(i)
print("Sentences padded.")

input_ids = torch.tensor(padded_sentences)
print("Input IDs Set")

with torch.no_grad():
    last_hidden_states = model(input_ids)[0]
    
print("Last hidden states Set")

sentence_embeddings = []
for i in range(len(sentences)):
    print(f"Appending sentence {i}")
    sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

print("Sentence embeddings complete.")

# Compute the similarity matrix
similarity_matrix = cosine_similarity(sentence_embeddings)

print("Similarity matrix complete.")

# Generate the summary
num_sentences = round(len(sentences)*.25)
summary_sentences = []
for i in range(num_sentences):
    sentence_scores = list(enumerate(similarity_matrix[i]))

sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

print("Sentence score completed.")

for i in range(num_sentences):
    summary_sentences.append(sentences[sentence_scores[i][0]])

print("Summary batch completed.")

summary = ' '.join(summary_sentences)

#     senators.at[i, "initiative_summary_en"] = summary

print(f"Summary complete!")

TypeError: object of type 'float' has no len()

In [214]:
for i, row in senators.iterrows():
    if len(row["initiatives_summary_dummy_en"]) >= 1:
        print(f"Working on summary {i} of {len(senators)}")
        sentences = sent_tokenize(row["initiatives_summary_dummy_en"])
        print(f"This senator has {len(sentences)} sentences.")
        tokenized_sentences = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]
        print("Finished sentence tokenization")
        max_len = 0
        for i in tokenized_sentences:
            if len(i) > max_len:
                max_len = len(i)

        padded_sentences = []
        for i in tokenized_sentences:
            while len(i) < max_len:
                i.append(0)
            padded_sentences.append(i)
        print("Sentences padded.")

        input_ids = torch.tensor(padded_sentences)

        with torch.no_grad():
            last_hidden_states = model(input_ids)[0]

        sentence_embeddings = []
        for i in range(len(sentences)):
            print(f"Appending sentence {i}")
            sentence_embeddings.append(torch.mean(last_hidden_states[i], dim=0).numpy())

        print("Sentence embeddings complete.")

        # Compute the similarity matrix
        similarity_matrix = cosine_similarity(sentence_embeddings)
        
        print("Similarity matrix complete.")

        # Generate the summary
        num_sentences = round(len(sentences)*.25)
        summary_sentences = []
        for i in range(num_sentences):
            sentence_scores = list(enumerate(similarity_matrix[i]))

        sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)
        
        print("Sentence score completed.")

        for i in range(num_sentences):
            summary_sentences.append(sentences[sentence_scores[i][0]])
        
        print("Summary batch completed.")

        summary = ' '.join(summary_sentences)

        senators.at[i, "initiative_summary_en"] = summary
        
        print(f"Summary {i} of {len(senators)} complete!")
    else:
        print(f"Senator number {i}, senator {row['senadores']} has no initiatives to summarize.")

Working on summary 0 of 126
{'It proposes to establish greater clarity in the subdivision of the ordinary commissions of Education, Culture and Science and Technology, suppressing the only current denomination of the ordinary commission of "Education, Culture, Science and Technology". It proposes to establish that the National Educational System will consider education in terms of privacy and protection of their personal data in the digital field, so it must be included in study plans and programs.'}
Working on summary 1 of 126
{'It proposes that this legal system regulates the prevention of environmental damage, for which it establishes economic sanctions and corrective measures provided as accessories to the repair or compensation of the damage caused to the environment when so determined. It proposes to establish that the authorities of the three levels of government must prevent, attend to and punish cases in which minors are affected by child hypersexualization. It also proposes t

{"It proposes to establish in current legislation that the access of women in situations of violence to social programs for the purpose of economic strengthening is guaranteed , psychological and legal. It proposes to establish that the National Policy on Equality between women and men will include the promotion of the participation and recognition of those women that operate in the health sector. Currently, childcare facilities meet certain objectives for children, which help them in their development, observing the provisions of the General Law on the Rights of Girls, Boys and Adolescents. It proposes to modify the treatment of crimes against freedom of expression, specifically when using violence to avoid the exercise of journalistic activity and when freedom of expression is hindered. It proposes to establish that the criteria contained in the statutes of national sports associations that favor the hiring of foreign professional athletes over nationals will be null. It proposes to 

{'-Unique and public register of beneficiaries. -Complementarities and coincidences with other federal programs. -Potential population, objective and eligibility mechanisms. -Matrix of Indicators for Results\r\n-Assigned budget. -Mechanisms for monitoring and accountability. -Impact and Results Measurements;\r\n-Technical Sheets. -Annexes. to whom they have requested it. Proposes that the calculation basis of the limits of campaign, raising it from 20 to 40%. It proposes to establish that any temporary occupation of a position must be founded and motivated before the competent administrative authority. -Attention mechanisms. For the implementation of these actions, Mexican doctors will be preferred to foreigners in equal circumstances. Creates the Fund for concurrent and complementary responsibilities of the states and municipalities in immigration and asylum and refuge matters. In the case of independent candidates, it proposes that the ceiling be increased from 10% to 20%. It propose

{'On the other hand, it prohibits the hiring of people under 15 years of age; Being older than 15 determines the employer\'s obligation to set days of no more than six hours. .Proposes to eliminate the criminal type of copulation with a person over fifteen years of age and under eighteen through their consent with deceit. Proposes that foreigners who enter the country with the predominant purpose of making use of health services, be fully charged the cost of these except in cases of emergencies . It proposes to establish that when requesting partial or total dismissal, authorization must be obtained of the Prosecutor or public servant to whom that power is delegated, who must have the authorization. On the other hand, those who disseminate ideas that violate human dignity will be imposed from one to three years in prison and from 50 to 100 UMAs. In addition to the above, it states that domestic workers residing at home, the working hours may not exceed 8 hours a day. It proposes establ

{'It proposes that the Federal Executive, the Chamber of Deputies, the Governors and Congresses of the States , and the Head of Government of Mexico City, must allocate and approve sufficient economic resources in all lThe budgets, without omitting any fiscal year, for the diagnosis, design, execution and evaluation of programs and actions for the social prevention of violence and crime. This proposal includes the following attributes for the formation of the body: a) Legal nature;\r\nb) Principles of action, and\r\nc) Attributions\r\nIt proposes to establish that the Congress of the Union analyzes and approves the status of the National Public Security Strategy on an annual basis, prior to the appearance of the head of the Secretariat of the branch. Enrique Peña Nieto, so that through the Ministry of Finance and Public Credit, he grants all the facilities to the states of the Republic of our northern and southern border so that they can access existing resources in the Fund for Border

...

{'e) Control and follow up on the biannual reports related to the general and statistical information on cases of violence against women that make up the National Bank of Data and Information on Cases of Violence against Women. c) Request reports from the Ministry of the Interior on the public policies implemented by the Federal Government to promote respect for the human rights of women, as well as their coordination with other levels of government on the matter. The head of the Ministry of Public Education is respectfully urged to present a detailed plan on comprehensive public policies that replace the guarantee of healthy eating for girls, boys and adolescents in vulnerable situations who are enrolled in basic education. As well as establishing rules for this type of registries, so as to guarantee the essential formalities of the procedure in the event of a possible impact on the human rights of migrants, by an act of authority. In addition, it proposes to update the powers of the 

{'In the event that there is not unanimity for the election of the presidency up to two times, a third round of voting will be carried out and the person who obtains the majority of votes will be elected. The formation of the Special Commission will be plural and will be integrated according to the proportionality of each parliamentary group with representation in the Senate of the Republic, each of which will have a substitute. It proposes to establish that in case of manifest urgency or extreme case regarding the integration of the Plenary of the Institute, the INAI Presidency may be elected by unanimous vote of at least four members of the Plenary. On the other hand, it is proposed that the Senate Justice Commission be obliged to submit the Criminal Prosecution Plan to the Plenary Session of the Republic Senate, said Plan will be approved by two thirds of the senators present. It proposes to establish that the Congress of the Union may meet in person or remotely for its ordinary per

{'It proposes to clearly state in the legislation that the competent authorities of the Federation, the federal entities, the Municipalities and the territorial demarcations of the Mexico City, will coordinate with each other or with institutions of the social and private sector to promote the optimal use of infrastructure solely of a public nature for physical culture and sports. SECOND.- The Senate of the Republic respectfully urges the Head of the Federal Executive Power to implement the necessary conditions so that private investment can establish itself in the sugarcane area of \u200b\u200bTenosique, Tabasco, in order to promote and trigger the economic growth of said region of the country. THIRD.- The Senate of the Republic respectfully exhorts the President of the Republic, through the Secretary of Agriculture and Rural Development, to summon the academic authorities and the union representatives of the Autonomous University of Chapingo and the National Institute of Forestry, Ag

{'It proposes include in the attributions of the Navy the organization, regulation and, where appropriate, guarantee the transit between the inhabited islands and the closest port on the continental shelf. It proposes to establish as one of the functions of the Board of Directors of the commissions, that of removing the Technical Secretary, with the approval of the commission. It proposes to establish that the Ministry of Foreign Affairs and the Ministry of Tourism participate in the tourist and commercial promotion of the country through Tourism Diplomacy. Determines that this right is a consequence of the human right to rest and leisure; and to the reasonable limitation of the duration of work and paid vacations. This program will establish the objectives, goals and lines of action aimed at promoting the development of tourist activity in these localities, taking advantage of their natural, cultural and recreational tourist attractions. It establishes the requirements to be able to a

KeyboardInterrupt: 

In [243]:
len(senators["initiatives_summary_dummy_en"][35])

145411